# Парсинг данных. Практика 2

Скачаем книги с сайта http://books.toscrape.com/catalogue/page-1.html

Мы хотим создать базу данных со следующей информацией:

* Ссылка на книгу
* Стоимость
* Название
* Оценка (кол-во звезд)

In [1]:
import requests  
from bs4 import BeautifulSoup

In [2]:
url = 'http://books.toscrape.com/catalogue/page-1.html'
response = requests.get(url)
response

<Response [200]>

In [3]:
soup = BeautifulSoup(response.content, 'html')

In [4]:
soup.find('title').text.strip()

'All products | Books to Scrape - Sandbox'

In [6]:
books = soup.find_all('article', {'class' : 'product_pod'})
books[0]

<article class="product_pod">
<div class="image_container">
<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [7]:
books[0].find('p', {'class': 'price_color'}).text

'£51.77'

In [8]:
books[0].find_all('a')[0].get('href')

'a-light-in-the-attic_1000/index.html'

In [10]:
'http://books.toscrape.com/catalogue/' + books[0].find_all('a')[0].get('href')

'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

In [12]:
books[0].find('h3').text # так тут указан неполное название книги, поэтому см ниже:

'A Light in the ...'

In [13]:
books[0].find('h3').find('a').get('title')

'A Light in the Attic'

In [14]:
books[7].find('p').get('class')[1]

'Three'

In [18]:
books[2].find('p').get('class')

['star-rating', 'One']

In [19]:
def get_page(p):
    url = f'http://books.toscrape.com/catalogue/page-{p}.html'
    response = requests.get(url)
    tree = BeautifulSoup(response.content, 'html.parser')
    books = tree.find_all('article', {'class' : 'product_pod'})
    
    info = []
    
    for book in books:
        info.append({'price': book.find('p', {'class': 'price_color'}).text,
                     'href': 'https://books.toscrape.com/catalogue/' + book.find('h3').find('a').get('href'),
                     'title': book.find('h3').find('a').get('title'),
                    'rating': book.find('p').get('class')[1]})
                     
    return info

In [2]:
from tqdm import tqdm #она импортирует прогресс бар и скорость (прикольная штучка)

In [21]:
import time
infa = []

for p in tqdm(range(1, 51)):
    try:
        infa.extend(get_page(p))
        time.sleep(3)
    except:
        print(p)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:52<00:00,  3.46s/it]


In [22]:
import pandas as pd

df = pd.DataFrame(infa)
print(df.shape)
df.head()

(1000, 4)


,price,href,title,rating
0,£51.77,https://books.toscrape.com/catalogue/a-light-i...,A Light in the Attic,Three
1,£53.74,https://books.toscrape.com/catalogue/tipping-t...,Tipping the Velvet,One
2,£50.10,https://books.toscrape.com/catalogue/soumissio...,Soumission,One
3,£47.82,https://books.toscrape.com/catalogue/sharp-obj...,Sharp Objects,Four
4,£54.23,https://books.toscrape.com/catalogue/sapiens-a...,Sapiens: A Brief History of Humankind,Five


In [7]:
df.to_csv('books_parsed.csv', index=False)

In [8]:
df.to_excel('books_parsed.xlsx', index=False)

## Парсинг – задание

По аналогии вам предстоит собрать данные с сайта https://quotes.toscrape.com/. 

Нужно получить цитату, автора и теги, связанные с этой цитатой. 

Напишите программу, где пользователь вводит тег, а ему выводятся все цитаты, у которых есть такой тег (например, 'truth').

In [3]:
# сохранить в словарь (ну можно и в пандас) но я так поняла лучше в словарь

url = 'https://quotes.toscrape.com/'
response = requests.get(url)
response

<Response [200]>

In [4]:
soup = BeautifulSoup(response.content, 'html')
soup.find('title').text.strip()

'Quotes to Scrape'

In [27]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Quotes to Scrape
  </title>
  <link href="/static/bootstrap.min.css" rel="stylesheet"/>
  <link href="/static/main.css" rel="stylesheet"/>
 </head>
 <body>
  <div class="container">
   <div class="row header-box">
    <div class="col-md-8">
     <h1>
      <a href="/" style="text-decoration: none">
       Quotes to Scrape
      </a>
     </h1>
    </div>
    <div class="col-md-4">
     <p>
      <a href="/login">
       Login
      </a>
     </p>
    </div>
   </div>
   <div class="row">
    <div class="col-md-8">
     <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
      <span class="text" itemprop="text">
       “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
      </span>
      <span>
       by
       <small class="author" itemprop="author">
        Albert Einstein
       </small>
       <a href="/author/Albert

In [28]:
soup.find_all('quote')

[]

In [30]:
soup.find_all('div', {'class' : 'quote'})

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

In [31]:
quot = soup.find_all('div', {'class' : 'quote'})
quot[0]

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

In [47]:
quot_text = quot[0].find('span').text
quot_text

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [48]:
quot_author = quot[0].find('small').text
quot_author

'Albert Einstein'

In [49]:
lst = quot[0].find_all('a', {'class':'tag'})
lst_tags = [x.text for x in lst]
lst_tags

['change', 'deep-thoughts', 'thinking', 'world']

In [ ]:
https://quotes.toscrape.com/page/50/

In [51]:
url2 = 'https://quotes.toscrape.com/page/50/'
response2 = requests.get(url2)
soup2 = BeautifulSoup(response2.content, 'html')
print(soup2.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Quotes to Scrape
  </title>
  <link href="/static/bootstrap.min.css" rel="stylesheet"/>
  <link href="/static/main.css" rel="stylesheet"/>
 </head>
 <body>
  <div class="container">
   <div class="row header-box">
    <div class="col-md-8">
     <h1>
      <a href="/" style="text-decoration: none">
       Quotes to Scrape
      </a>
     </h1>
    </div>
    <div class="col-md-4">
     <p>
      <a href="/login">
       Login
      </a>
     </p>
    </div>
   </div>
   <div class="row">
    <div class="col-md-8">
     No quotes found!
     <nav>
      <ul class="pager">
       <li class="previous">
        <a href="/page/49/">
         <span aria-hidden="true">
          ←
         </span>
         Previous
        </a>
       </li>
      </ul>
     </nav>
    </div>
    <div class="col-md-4 tags-box">
     <h2>
      Top Ten tags
     </h2>
     <span class="tag-item">
      <a class="tag" href="/tag/love

In [66]:
soup.find_all('div', {'class':'col-md-8'})[1].text.strip()

'No quotes found!\n\n    \n\n\n← Previous'

In [74]:
'https://quotes.toscrape.com' + soup.find('nav').find('a').get('href')

'https://quotes.toscrape.com/page/2/'

In [14]:
url = 'https://quotes.toscrape.com/page/1/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html')


In [17]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Quotes to Scrape
  </title>
  <link href="/static/bootstrap.min.css" rel="stylesheet"/>
  <link href="/static/main.css" rel="stylesheet"/>
 </head>
 <body>
  <div class="container">
   <div class="row header-box">
    <div class="col-md-8">
     <h1>
      <a href="/" style="text-decoration: none">
       Quotes to Scrape
      </a>
     </h1>
    </div>
    <div class="col-md-4">
     <p>
      <a href="/login">
       Login
      </a>
     </p>
    </div>
   </div>
   <div class="row">
    <div class="col-md-8">
     <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
      <span class="text" itemprop="text">
       “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
      </span>
      <span>
       by
       <small class="author" itemprop="author">
        Albert Einstein
       </small>
       <a href="/author/Albert

In [40]:
response = requests.get('https://quotes.toscrape.com')
soup = BeautifulSoup(response.content, 'html')

# 'https://quotes.toscrape.com' + 
soup.find('nav').find('li', {'class': 'next'}).find('a').get('href')

'/page/2/'

In [39]:
response = requests.get('https://quotes.toscrape.com/page/1/')
soup = BeautifulSoup(response.content, 'html')

# 'https://quotes.toscrape.com' + 
soup.find('nav').find('li', {'class': 'next'}).find('a').get('href')

'/page/2/'

In [41]:
url = 'https://quotes.toscrape.com'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html')

        
infa = [] 

i = 0
while 'No quotes found!' not in soup.find_all('div', {'class':'col-md-8'})[1].text:
    try:
        print(i+1, 'get it', url)
        i+=1

        quots = soup.find_all('div', {'class' : 'quote'})

        info = []

        for quot in quots:
            lst = quot[0].find_all('a', {'class':'tag'})
            lst_tags = [x.text for x in lst]
            info.append({'quote': quot.find('span').text
                        ,'author': quot.find('small').text
                        ,'tags': lst_tags})
        
        print(soup.find('nav').find('li', {'class': 'next'}).find('a').get('href'))
        url = 'https://quotes.toscrape.com' + soup.find('nav').find('li', {'class': 'next'}).find('a').get('href') #next
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html')
        
    except:
        print(i, url)
        
    infa.extend(info)
    time.sleep(3)
    
print('stop')

1 get it https://quotes.toscrape.com
1 https://quotes.toscrape.com
2 get it https://quotes.toscrape.com
2 https://quotes.toscrape.com


KeyboardInterrupt: 

In [36]:
infa

[]